<a href="https://colab.research.google.com/github/Elvira-Zainulina/ml_MRI_age_prediction/blob/master/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torchio

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 42.5MB 85kB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import os
from ml_utils.uploading import unzip_imgs
from ml_utils.uploading import upload_raw_data
import pickle
from ml_utils.data import S500MRI_Dataset
from ml_utils.preprocessing import get_hist_landmarks
RND_STATE = 42

Define pathes to all essenatial data:

In [0]:
path = r"drive/My Drive/Colab Notebooks/ml-project/"

archive_path = os.path.join(path, 'tmp')
images_path = os.path.join(path, 'images')
table_path = os.path.join(path, 'data', 'table_with_ages.csv')

# Data loading

### Processing raw uploaded archives


After the downloading of the S900 dataset to the folder ```archive_path```, it containes a lof of archives each of which includes data describing one object (patient). To make working with the dataset easier, the archives can be unzipped in the folder ```images_path``` with the following function ```unzip_imgs```. After that each folder in the ```images_path``` will contain folder named by object id with the T1 weighted MRI (```img_path/object_id/T1w/T1w_acpc_dc_restore_brain.nii.gz```).

In [0]:
unzip_imgs(archive_path, images_path)

Unzipped 185442_3T_Structural_preproc.zip
Unzipped 181131_3T_Structural_preproc.zip
Unzipped 181232_3T_Structural_preproc.zip
Unzipped 182739_3T_Structural_preproc.zip
Unzipped 182840_3T_Structural_preproc.zip
Unzipped 183034_3T_Structural_preproc.zip
Unzipped 185139_3T_Structural_preproc.zip


### Train-test dataset division

As the data is quite large, it will be difficult to work directly with the data. Therefore, we firstly divide the data into train, validation and test sets using object ids and labels. Sizes of validation and test sets are set not too big because we have limited amount of data.

In [19]:
#The cell can be not executed, as object ids for train, validation and test
#sets are provided in files train_names.npy, validation_names.npy, test_names.npy

# Size of the whole dataset: 899
# --------------------------------------
# Size of the train dataset: 800
# Size of the test dataset: 99
# --------------------------------------
# Size of the train part of the train dataset: 700
# Size of the validation part of the train dataset: 100

# from sklearn.model_selection import train_test_split

# df = pd.read_csv(table_path)

# names = []
# labels = []
# for f in os.listdir(images_path):
#     f = int(f)
#     names.append(f)
#     labels.append(df.Age.values[df.Subject == f][0])

# print('Size of the whole dataset:', len(names))
# print('--------------------------------------')

# (train_names, test_names,
#  train_labels, test_labels) = train_test_split(names, labels, test_size=0.11, 
#                                                random_state=RND_STATE)
# print('Size of the train dataset:', len(train_names))
# print('Size of the test dataset:', len(test_names))
# np.save(os.path.join(path, 'data', 'full_train_names.npy'), train_names)
# np.save(os.path.join(path, 'data', 'test_names.npy'), test_names)

# train_names, val_names = train_test_split(train_names, test_size=0.125, 
#                                           stratify=train_labels,
#                                           random_state=RND_STATE)
# print('Size of the train part of the train dataset:', len(train_names))
# print('Size of the validation part of the train dataset:', len(val_names))
# np.save(os.path.join(path, 'data', 'train_names.npy'), train_names)
# np.save(os.path.join(path, 'data', 'val_names.npy'), val_names)

Size of the whole dataset: 899
--------------------------------------
Size of the train dataset: 800
Size of the test dataset: 99
Size of the train part of the train dataset: 700
Size of the validation part of the train dataset: 100


Upload saved division of object ids into train, validation and test sets:

In [0]:
train_names = np.load(os.path.join(path, 'data', 'train_names.npy'))
test_names = np.load(os.path.join(path, 'data', 'test_names.npy'))
val_names = np.load(os.path.join(path, 'data', 'val_names.npy'))

### Data loading dapanding on the defined division

As it was mentioned before, it is difficult to work with 3D MRIs directly because of the large size of MRIs. The ```torchio``` library allows to overcome this problem. It allows not to upload all images at once and upload it only when the object being processed. 

Upload data from the ```image_path``` folder with all MRIs using division of the ids into train, validation, test sets. Labels (ages) and additional information (genders) are taken from the corresponding table data (```table_path```). Uploaded MRIs are of the type ```torchio.Subject```.

In [22]:
%%time
train_data = upload_raw_data(images_path, table_path, train_names)
val_data = upload_raw_data(images_path, table_path, val_names)
test_data = upload_raw_data(images_path, table_path, test_names)

CPU times: user 1.43 s, sys: 107 ms, total: 1.54 s
Wall time: 2min 24s


Save the data for emergency case (uncomment the following cell)

In [0]:
#   with open(os.path.join(path, 'data', 'train_data.pickle'), 'wb') as f:
#     pickle.dump(train_data, f)

# with open(os.path.join(path, 'data', 'val_data.pickle'), 'wb') as f:
#     pickle.dump(val_data, f)

# with open(os.path.join(path, 'data', 'test_data.pickle'), 'wb') as f:
#     pickle.dump(test_data, f)

# Data preprocessing


Upload saved train, validation and test data.

In [0]:
with open(os.path.join(path, 'data', 'train_data.pickle'), 'rb') as f:
    train_data = pickle.load(f)

with open(os.path.join(path, 'data', 'val_data.pickle'), 'rb') as f:
    val_data = pickle.load(f)

with open(os.path.join(path, 'data', 'test_data.pickle'), 'rb') as f:
    test_data = pickle.load(f)

The ```torchio``` library proposes several method for MRI preprocessing. One of the preprocessing functions based on the Histogram Normalization. To carry out his normalization it needs landmarks. This landmarks are obtained in the following cell from the train and validation datasets:

In [26]:
landmarks_path = os.path.join(path, 'data', 'landmarks900S.npy')

landmarks = get_hist_landmarks(images_path, landmarks_path,
                               np.append(train_names, val_names))


100%|██████████| 800/800 [16:18<00:00,  1.22s/it]


Compose preprocessing methods from ```torchio```  that will be applied to the MRIs in order to make proper image normalization and custom preprocessing functions that will be used to make MRI images of size $128\times 128\times 128$.

In [0]:
from torchio.transforms import (
    ToCanonical,
    ZNormalization,
    HistogramStandardization,
)

from torchvision.transforms import Compose
from ml_utils.transforms import ResizeCrop, BoundsCrop

In [0]:
transform = Compose([
    BoundsCrop(),                            
    HistogramStandardization(landmarks_dict={'MRI': landmarks}),
    ZNormalization(masking_method=ZNormalization.mean),
    ToCanonical(),
    ResizeCrop(128)
])

Define S500MRI train, validation and test datasets (```torchio.ImagesDataset``` that were fixed in order to make it possible to use for age classifiaction problem with additional information about object gender). This datasets can be directly used for models training and, therefore, additional ```torchio``` methods for data augmentation can be applied. But usage of these datasets during models training makes makes the training slower because of the necessity to load images from the storage and process it at every step. Because of the time and limited computational resources, it was decided to apply only preprocessing methods from ```torchio``` and store obtained processed data.

In [0]:
train_dataset = S500MRI_Dataset(**train_data, transform=transform)
val_dataset = S500MRI_Dataset(**val_data, transform=transform)
test_dataset = S500MRI_Dataset(**test_data, transform=transform)

In [0]:
from ml_utils.preprocessing import process_dataset

Process all data and store it.

In [31]:
_ = process_dataset(train_dataset, os.path.join(path, 'data', 'train_dataset.npy'))

In [32]:
_ = process_dataset(val_dataset, os.path.join(path, 'data', 'val_dataset.npy'))

In [33]:
_ = process_dataset(test_dataset, os.path.join(path, 'data', 'test_dataset.npy'))